In [1]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import os
import glob
import random
import pickle
import pandas as pd
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, precision_score, f1_score, confusion_matrix, recall_score
import matplotlib.pyplot as plt
import seaborn as sns
from tensorflow.keras import layers, models, optimizers
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import tensorflow as tf

In [2]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import os
import glob
import random
import pickle
import pandas as pd
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, precision_score, f1_score, confusion_matrix, recall_score

In [3]:
# Getting path of folders and their images
path_to_images = 'H:\FINAL PROJECT DATASET\Dataset'

img_folders = []
for entry in os.scandir(path_to_images):
    if entry.is_dir():
        img_folders.append(entry.path)
img_folders

['H:\\FINAL PROJECT DATASET\\Dataset\\Mild_Demented',
 'H:\\FINAL PROJECT DATASET\\Dataset\\Moderate_Demented',
 'H:\\FINAL PROJECT DATASET\\Dataset\\Non_Demented',
 'H:\\FINAL PROJECT DATASET\\Dataset\\Very_Mild_Demented']

In [4]:
# Creating a dictionary of images and their labels from each folder path

img_dict = {}
for folder in img_folders:
    img_dict[folder] = []
    for entry in os.scandir(folder):
        if entry.is_file():
            img_dict[folder].append(entry.path)
img_dict

{'H:\\FINAL PROJECT DATASET\\Dataset\\Mild_Demented': ['H:\\FINAL PROJECT DATASET\\Dataset\\Mild_Demented\\mild.jpg',
  'H:\\FINAL PROJECT DATASET\\Dataset\\Mild_Demented\\mild_10.jpg',
  'H:\\FINAL PROJECT DATASET\\Dataset\\Mild_Demented\\mild_100.jpg',
  'H:\\FINAL PROJECT DATASET\\Dataset\\Mild_Demented\\mild_101.jpg',
  'H:\\FINAL PROJECT DATASET\\Dataset\\Mild_Demented\\mild_102.jpg',
  'H:\\FINAL PROJECT DATASET\\Dataset\\Mild_Demented\\mild_103.jpg',
  'H:\\FINAL PROJECT DATASET\\Dataset\\Mild_Demented\\mild_104.jpg',
  'H:\\FINAL PROJECT DATASET\\Dataset\\Mild_Demented\\mild_105.jpg',
  'H:\\FINAL PROJECT DATASET\\Dataset\\Mild_Demented\\mild_106.jpg',
  'H:\\FINAL PROJECT DATASET\\Dataset\\Mild_Demented\\mild_107.jpg',
  'H:\\FINAL PROJECT DATASET\\Dataset\\Mild_Demented\\mild_108.jpg',
  'H:\\FINAL PROJECT DATASET\\Dataset\\Mild_Demented\\mild_109.jpg',
  'H:\\FINAL PROJECT DATASET\\Dataset\\Mild_Demented\\mild_11.jpg',
  'H:\\FINAL PROJECT DATASET\\Dataset\\Mild_Demented\\mi

In [5]:
# Creating a dataframe of images and their labels

img_df = pd.DataFrame(columns=["image", "label"])
for folder in img_dict:
    for img in img_dict[folder]:
        img_df = img_df.append({"image": img, "label": folder.split("\\")[-1]}, ignore_index=True)
img_df

C:\Users\hp\AppData\Local\Temp\ipykernel_8864\753985834.py:6: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  img_df = img_df.append({"image": img, "label": folder.split("\\")[-1]}, ignore_index=True)
C:\Users\hp\AppData\Local\Temp\ipykernel_8864\753985834.py:6: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  img_df = img_df.append({"image": img, "label": folder.split("\\")[-1]}, ignore_index=True)
C:\Users\hp\AppData\Local\Temp\ipykernel_8864\753985834.py:6: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  img_df = img_df.append({"image": img, "label": folder.split("\\")[-1]}, ignore_index=True)
C:\Users\hp\AppData\Local\Temp\ipykernel_8864\753985834.py:6: FutureWarning: The frame.append method is deprecated and will be removed f

,image,label
0,H:\FINAL PROJECT DATASET\Dataset\Mild_Demented...,Mild_Demented
1,H:\FINAL PROJECT DATASET\Dataset\Mild_Demented...,Mild_Demented
2,H:\FINAL PROJECT DATASET\Dataset\Mild_Demented...,Mild_Demented
3,H:\FINAL PROJECT DATASET\Dataset\Mild_Demented...,Mild_Demented
4,H:\FINAL PROJECT DATASET\Dataset\Mild_Demented...,Mild_Demented
...,...,...
6395,H:\FINAL PROJECT DATASET\Dataset\Very_Mild_Dem...,Very_Mild_Demented
6396,H:\FINAL PROJECT DATASET\Dataset\Very_Mild_Dem...,Very_Mild_Demented
6397,H:\FINAL PROJECT DATASET\Dataset\Very_Mild_Dem...,Very_Mild_Demented
6398,H:\FINAL PROJECT DATASET\Dataset\Very_Mild_Dem...,Very_Mild_Demented


In [6]:
# Shuffling the dataframe

img_df = shuffle(img_df)
img_df

,image,label
1172,H:\FINAL PROJECT DATASET\Dataset\Non_Demented\...,Non_Demented
1129,H:\FINAL PROJECT DATASET\Dataset\Non_Demented\...,Non_Demented
163,H:\FINAL PROJECT DATASET\Dataset\Mild_Demented...,Mild_Demented
2059,H:\FINAL PROJECT DATASET\Dataset\Non_Demented\...,Non_Demented
3217,H:\FINAL PROJECT DATASET\Dataset\Non_Demented\...,Non_Demented
...,...,...
937,H:\FINAL PROJECT DATASET\Dataset\Moderate_Deme...,Moderate_Demented
5457,H:\FINAL PROJECT DATASET\Dataset\Very_Mild_Dem...,Very_Mild_Demented
4676,H:\FINAL PROJECT DATASET\Dataset\Very_Mild_Dem...,Very_Mild_Demented
698,H:\FINAL PROJECT DATASET\Dataset\Mild_Demented...,Mild_Demented


In [7]:
#Encoding the labels

encode_map = {"Non_Demented" : 0, "Very_Mild_Demented" : 1, "Mild_Demented" : 2, "Moderate_Demented" : 3}
img_df["label"] = img_df["label"].map(encode_map)
img_df

,image,label
1172,H:\FINAL PROJECT DATASET\Dataset\Non_Demented\...,0
1129,H:\FINAL PROJECT DATASET\Dataset\Non_Demented\...,0
163,H:\FINAL PROJECT DATASET\Dataset\Mild_Demented...,2
2059,H:\FINAL PROJECT DATASET\Dataset\Non_Demented\...,0
3217,H:\FINAL PROJECT DATASET\Dataset\Non_Demented\...,0
...,...,...
937,H:\FINAL PROJECT DATASET\Dataset\Moderate_Deme...,3
5457,H:\FINAL PROJECT DATASET\Dataset\Very_Mild_Dem...,1
4676,H:\FINAL PROJECT DATASET\Dataset\Very_Mild_Dem...,1
698,H:\FINAL PROJECT DATASET\Dataset\Mild_Demented...,2


In [8]:
# Creating X and y by reading images and their labels

X = []
y = []

for img_path, label in zip(img_df["image"].values, img_df["label"].values):
    img = cv2.imread(img_path)
    img = cv2.resize(img, (224, 224))
    X.append(img)
    y.append(label)
X = np.array(X)
y = np.array(y)

In [9]:
X.shape, y.shape

((6400, 224, 224, 3), (6400,))

In [10]:
img_df.isna().sum()

image    0
label    0
dtype: int64

In [12]:
# Set seed for reproducibility
seed = 42
np.random.seed(seed)
tf.random.set_seed(seed)

In [13]:
# Splitting the data into train, validation and test

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, stratify=y, random_state=seed)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, stratify=y_train)

In [14]:
# Define the DenseNet model
base_model = tf.keras.applications.DenseNet121(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

29084464/29084464 [==============================] - 6s 0us/step


In [15]:
# Freeze the layers in the base model
for layer in base_model.layers:
    layer.trainable = False

In [16]:
#Build your own model on top of DenseNet
model = tf.keras.models.Sequential()
model.add(base_model)
model.add(tf.keras.layers.GlobalAveragePooling2D())
model.add(tf.keras.layers.Dense(256, activation='relu'))
model.add(tf.keras.layers.Dropout(0.5))
model.add(tf.keras.layers.Dense(1, activation='sigmoid'))


In [17]:
# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [20]:
# Train the model
model.fit(X_train, y_train, epochs=20, batch_size=32, validation_split=0.1, shuffle=True)

Epoch 1/20
87/87 [==============================] - 423s 5s/step - loss: 0.9415 - accuracy: 0.4902 - val_loss: 0.0212 - val_accuracy: 0.5000
Epoch 2/20
87/87 [==============================] - 406s 5s/step - loss: 0.2544 - accuracy: 0.5340 - val_loss: -0.0280 - val_accuracy: 0.5130
Epoch 3/20
87/87 [==============================] - 402s 5s/step - loss: 0.2499 - accuracy: 0.5235 - val_loss: -0.1405 - val_accuracy: 0.5584
Epoch 4/20
87/87 [==============================] - 411s 5s/step - loss: 0.0923 - accuracy: 0.5289 - val_loss: -0.0236 - val_accuracy: 0.5714
Epoch 5/20
87/87 [==============================] - 406s 5s/step - loss: 0.0883 - accuracy: 0.5427 - val_loss: -0.3254 - val_accuracy: 0.5812
Epoch 6/20
87/87 [==============================] - 405s 5s/step - loss: -0.0607 - accuracy: 0.5416 - val_loss: -0.3287 - val_accuracy: 0.5714
Epoch 7/20
87/87 [==============================] - 401s 5s/step - loss: -0.2882 - accuracy: 0.5423 - val_loss: 0.1212 - val_accuracy: 0.5455
Epoch 

In [21]:
# Evaluate the model on the test set
y_pred = (model.predict(X_test) > 0.5).astype(int)

80/80 [==============================] - 319s 4s/step


In [22]:
# Calculate and print metrics
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, average="weighted")
recall = recall_score(y_test, y_pred, average="weighted")
f1 = f1_score(y_test, y_pred, average="weighted")
conf_matrix = confusion_matrix(y_test, y_pred)

C:\Users\hp\AppData\Roaming\Python\Python310\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [23]:
print(f"Accuracy: {accuracy}")
print(f"Precision: {precision}")
print(f"Recall: {recall}")
print(f"F1 Score: {f1}")
print("Confusion Matrix:")
print(conf_matrix)

Accuracy: 0.55078125
Precision: 0.5329982414600035
Recall: 0.55078125
F1 Score: 0.5158812955922458
Confusion Matrix:
[[714 566   0   0]
 [200 696   0   0]
 [ 17 341   0   0]
 [  0  26   0   0]]
